# Imports

In [1]:
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots
import tldextract
tqdm.pandas()
from pandarallel import pandarallel
import json
from utils import *

pandarallel.initialize(progress_bar=True, nb_workers=32)

# plt.style.use("science")
plt.style.use(["science", "ieee", "std-colors"])
mpl.use("pgf")

params = {
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False,
    "pgf.texsystem": "xelatex",
    "pgf.preamble": "\n".join(
        [
            r"\usepackage{fontspec, xeCJK}",
            r"\setmainfont{Times New Roman}",
            # r"\setCJKmainfont{SimSong}",
            # r"\setCJKsansfont{Hei}",
        ]
    ),
}
mpl.rcParams.update(params)
plt.rcParams.update({"figure.dpi": "100"})

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Load Data

In [2]:
%load_ext google.cloud.bigquery

/opt/miniconda3/envs/wiretapping/lib/python3.12/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [3]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "javascript-wiretapping-38b1da3ce1a3.json"

In [4]:
%%bigquery df_event_listeners --project=javascript-wiretapping
SELECT
    site, event_type, event, event_time, init_invoke, stack
FROM 
measurement_imc_final.event_listeners

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
%%bigquery df_interactions --project=javascript-wiretapping
SELECT *
FROM 
measurement_imc_final.interactions

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
%%bigquery df_callstacks --project=javascript-wiretapping
SELECT site, init_invoke, event_type, stack, timestamp, request_url
FROM 
measurement_imc_final.callstacks
WHERE
event_type like "%onkey%"

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
%%bigquery df_requests --project=javascript-wiretapping
SELECT site, url, method, request_time, after_interaction, data_leak, time, data_leak_type, request_call_stack
FROM 
measurement_imc_final.requests

Query is running:   0%|          |

Downloading:   0%|          |

In [8]:
%%bigquery df_sitecats --project=javascript-wiretapping
SELECT * FROM javascript-wiretapping.measurement_imc_final.site_cats

Executing query with job ID: c67367f7-7aea-4b30-a33e-49b55b31830e
Query executing: 0.61s


ERROR:
 404 Not found: Table javascript-wiretapping:measurement_imc_final.site_cats was not found in location US; reason: notFound, message: Not found: Table javascript-wiretapping:measurement_imc_final.site_cats was not found in location US

Location: US
Job ID: c67367f7-7aea-4b30-a33e-49b55b31830e



# Process Data

In [9]:
df_event_listeners["url"] = df_event_listeners["stack"].parallel_apply(
    find_first_url_in_callstack
)

In [10]:
df_event_listeners["filename"] = df_event_listeners["url"].parallel_apply(
    get_javascript_filename
)

In [11]:
df_requests['calling_script'] = df_requests.request_call_stack.parallel_apply(find_script_in_callstack_json)

In [12]:
df_requests['calling_script_filename'] = df_requests['calling_script'].parallel_apply(get_javascript_filename)

In [13]:
df_callstacks['url'] = df_callstacks['stack'].parallel_apply(
    find_first_url_in_callstack
)

In [14]:
df_callstacks["filename"] = df_callstacks["url"].parallel_apply(get_javascript_filename)

In [15]:
df_event_listeners["script_domain"] = df_event_listeners["url"].parallel_apply(
    get_domain
)

In [16]:
df_event_listeners['site_domain'] = df_event_listeners.site.parallel_apply(get_domain)

In [17]:
df_callstacks['script_domain'] = df_callstacks['url'].parallel_apply(get_domain)

In [18]:
df_callstacks['site_domain'] = df_callstacks['site'].parallel_apply(get_domain)

In [19]:
df_interactions['site_domain'] = df_interactions['site_url'].parallel_apply(get_domain)

In [20]:
df_requests['site_domain'] = df_requests['site'].parallel_apply(get_domain)

In [21]:
df_requests['request_domain'] = df_requests['url'].parallel_apply(get_domain)

In [22]:
df_requests['calling_script_domain'] = df_requests['calling_script'].parallel_apply(get_domain)

In [23]:
df_event_listeners.drop('event', axis=1, inplace=True)

In [24]:
df_callstacks.rename(columns={'timestamp': 'event_time'}, inplace=True)

In [25]:
df_event_listeners = pd.concat([df_event_listeners, df_callstacks])

In [26]:
df_event_listeners.event_type.replace('onkeydown', 'keydown', inplace=True)
df_event_listeners.event_type.replace('onkeyup', 'keyup', inplace=True)
df_event_listeners.event_type.replace('onkeypress', 'keypress', inplace=True)

/tmp/ipykernel_1946816/3662881018.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_event_listeners.event_type.replace('onkeydown', 'keydown', inplace=True)
/tmp/ipykernel_1946816/3662881018.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].m

In [27]:
df_event_listeners['script_entity'] = df_event_listeners['script_domain'].parallel_apply(get_entity)

In [28]:
df_event_listeners['site_entity'] = df_event_listeners['site_domain'].parallel_apply(get_entity)

In [29]:
df_interactions['site_entity'] = df_interactions['site_domain'].parallel_apply(get_entity)

In [30]:
df_requests['site_domain'] = df_requests['site'].parallel_apply(get_domain)

In [31]:
df_requests['request_entity'] = df_requests['request_domain'].parallel_apply(get_entity)

In [32]:
df_requests['calling_script_entity'] = df_requests['calling_script_domain'].parallel_apply(get_entity)

# Save Data To Parquet

In [33]:
# download_lists("filterlists")
filterlists, filterlist_rules = create_filterlist_rules("filterlists")

In [56]:
df_event_listeners['event_time'] = pd.to_datetime(
    df_event_listeners['event_time'],
    utc=True,
    errors='coerce',    # bad or out-of-range → NaT
    format='mixed'      # try inferring formats per-row
)

In [57]:
df_event_listeners.to_parquet("df_event_listeners.parquet", engine="pyarrow", index=False)

In [39]:
df_interactions.head()

,site_id,site_url,start_time,end_time,interaction_type,time_start,time_end,measurement,site_domain,site_entity
0,1050_4,https://www.ilovepdf.com/add_pdf_page_number,2025-04-25 16:08:31.897000+00:00,2025-04-25 16:08:32.338000+00:00,fill_form_range,16:08:31.897000,16:08:32.338000,25k_chunk_1.db,ilovepdf.com,ILOVEPDF S.L.
1,1050_4,https://www.ilovepdf.com/add_pdf_page_number,2025-04-25 16:08:43.604000+00:00,2025-04-25 16:08:43.965000+00:00,fill_form_range,16:08:43.604000,16:08:43.965000,25k_chunk_1.db,ilovepdf.com,ILOVEPDF S.L.
2,1079_5,https://letterboxd.com/about/,2025-04-25 13:12:57.892000+00:00,2025-04-25 13:12:58.580000+00:00,fill_form_range,13:12:57.892000,13:12:58.580000,25k_chunk_1.db,letterboxd.com,Cactuslab Limited
3,1079_6,https://letterboxd.com/afi/,2025-04-25 13:13:52.848000+00:00,2025-04-25 13:13:53.368000+00:00,fill_form_range,13:13:52.848000,13:13:53.368000,25k_chunk_1.db,letterboxd.com,Cactuslab Limited
4,1079_7,https://letterboxd.com/afi/story/spotlight-on-...,2025-04-25 13:14:40.610000+00:00,2025-04-25 13:14:41.575000+00:00,fill_form_range,13:14:40.610000,13:14:41.575000,25k_chunk_1.db,letterboxd.com,Cactuslab Limited


In [ ]:
# if they’re already datetime64[us, UTC], this will up-cast to ns and preserve UTC
df_interactions["start_time"] = df_interactions["start_time"].astype(
    "datetime64[ns, UTC]"
)
df_interactions["end_time"] = df_interactions["end_time"].astype("datetime64[ns, UTC]")

# drop unneeded columns
df_interactions.drop(
    ["time_end", "time_start"],
    axis=1,
    inplace=True,
)

df_interactions.to_parquet("df_interactions.parquet", engine="pyarrow", index=False)

In [ ]:
df_requests['request_time'] = pd.to_datetime(df_requests['request_time'], utc=True)

In [47]:
df_requests.to_parquet("df_requests.parquet", engine="pyarrow", index=False)

In [51]:
df_callstacks['event_time'] = pd.to_datetime(
    df_callstacks['event_time'],
    utc=True,           # or drop this if your data has no timezone
)


In [52]:
df_callstacks.to_parquet("df_callstacks.parquet", engine="pyarrow", index=False)

# Read saved data

In [2]:
df_event_listeners = pd.read_parquet("df_event_listeners.parquet", engine="pyarrow")

In [3]:
df_interactions = pd.read_parquet("df_interactions.parquet", engine="pyarrow")

In [4]:
import pyarrow as pa
import pyarrow.parquet as pq

# 1) Read the Parquet file _without_ pulling in pandas metadata
table = pq.read_table("df_requests.parquet", use_pandas_metadata=False)

# 2) (Optional) Strip _all_ schema metadata to be 100% sure
table = table.replace_schema_metadata({})

# 3) Drop your unwanted 'time' column (if it still exists)
if "time" in table.schema.names:
    idx = table.schema.get_field_index("time")
    table = table.remove_column(idx)

# 4) Convert to a DataFrame
df_requests = table.to_pandas()

In [4]:
df_requests = pd.read_parquet("df_requests.parquet", engine="pyarrow")

TypeError: data type 'dbtime' not understood

In [5]:
df_callstacks = pd.read_parquet("df_callstacks.parquet", engine="pyarrow")

# Event Listener Statistics

In [6]:
n_sites = len(list(set(df_event_listeners.site_domain.unique().tolist() + df_requests.site_domain.unique().tolist() + df_interactions.site_domain.unique().tolist() + df_callstacks.site_domain.unique().tolist())))

In [7]:
print("Total number of sites: ", n_sites)

Total number of sites:  14471


In [8]:
print("Total types of event listeners:", len(df_event_listeners.event_type.unique().tolist()))

Total types of event listeners: 12781


In [9]:
df_temp = df_event_listeners[
    ["site_domain", "event_type"]
].drop_duplicates().event_type.value_counts().reset_index()[:20]
df_temp["count_percentage"] = df_temp["count"] / n_sites * 100
df_temp[['event_type','count_percentage']].to_latex(
    "event_types.tex",
    column_format="lr",
    float_format="%.2f",
    index=False,
    escape=False,                                    # allow raw LaTeX in your cells
    formatters={
        'event_type': lambda x: f'\\texttt{{{x}}}'  # wrap each entry in \texttt{…}
    }
)

In [10]:
del df_temp

In [11]:
print("Event listeners set by first-party scripts:", df_event_listeners[df_event_listeners.site_entity == df_event_listeners.script_entity][
    ["site_domain", "event_type", "script_domain"]
].drop_duplicates().shape[0] / df_event_listeners[
    ["site_domain", "event_type", "script_domain"]
].drop_duplicates().shape[
    0
] * 100)

Event listeners set by first-party scripts: 18.70075032957012


In [12]:
print("Event listeners set by third-party scripts:", df_event_listeners[df_event_listeners.site_entity != df_event_listeners.script_entity][
    ["site_domain", "event_type", "script_domain"]
].drop_duplicates().shape[0] / df_event_listeners[
    ["site_domain", "event_type", "script_domain"]
].drop_duplicates().shape[
    0
] * 100)

Event listeners set by third-party scripts: 81.29924967042987


In [13]:
print("Percentage of sites where at least one event listener is present:", df_event_listeners[
    ["site_domain", "event_type"]
].site_domain.nunique()/n_sites * 100)

Percentage of sites where at least one event listener is present: 91.47951074562918


In [14]:
print("Average number of event listeners installed per website:", df_event_listeners[['site_domain', 'event_type', 'script_domain']].drop_duplicates().shape[0]/n_sites)

Average number of event listeners installed per website: 48.802639762283185


In [15]:
def get_top_three_most_used_events(script_domain):
    return (
        df_event_listeners[df_event_listeners.script_domain == script_domain]
        .event_type.value_counts()
        .head(3)
        .index.tolist()
    )

def get_top_three_most_used_scripts(script_domain):
    return (
        df_event_listeners[(df_event_listeners.script_domain == script_domain) & (df_event_listeners.filename.str.contains('.js'))]
        .filename.value_counts()
        .head(3)
        .index.tolist()
    )


df_event_listeners_stats = df_event_listeners[
    df_event_listeners.site_entity != df_event_listeners.script_entity
][["site_domain", "script_domain", "event_type"]]
df_event_listeners_stats.drop_duplicates(inplace=True)

df_script_stats = (
    df_event_listeners_stats[["script_domain", "site_domain"]]
    .drop_duplicates()
    .groupby("script_domain")
    .size()
    .reset_index(name="count")
)

df_top_scripts = (
    df_script_stats.sort_values(by="count", ascending=False)
    .head(20)
    .script_domain.tolist()
)

df_script_info = (
    df_script_stats.sort_values(by="count", ascending=False)
    .reset_index(drop=True)
    .head(20)
    .rename(columns={"script_domain": "Script Domain", "count": "Number of Sites"})
)

df_script_info["event_types"] = df_script_info["event_types"] = df_script_info[
    "Script Domain"
].apply(lambda x: ", ".join(get_top_three_most_used_events(x)))

df_script_info["scripts_used"] = df_script_info["Script Domain"].apply(
    lambda x: ", ".join(get_top_three_most_used_scripts(x))
)

df_script_info["Percentage of Sites"] = (
    df_script_info["Number of Sites"] / n_sites * 100
)

# round to 2 decimal places
df_script_info["Percentage of Sites"] = df_script_info["Percentage of Sites"].round(2)

df_script_info[
    ["Script Domain", "Percentage of Sites", "event_types", "scripts_used"]
].to_latex("top_scripts.tex", index=False, column_format="lrr", float_format="%.2f")

# Wiretapping Stats

In [16]:
wiretapping_events = [
    "keypress",
    "keypressed",
    "keyup",
    "keydown",
]
df_event_listeners_wiretappers = df_event_listeners[
    (df_event_listeners.event_type.isin(wiretapping_events)) & (df_event_listeners.site_entity != df_event_listeners.script_entity)
]

df_request_exfils = df_requests[df_requests.data_leak == True]

df_request_exfiltrators = df_request_exfils[
    ~df_request_exfils.calling_script_domain.isna()
][
    [
        "site",
        "calling_script_filename",
        "calling_script_domain",
        "calling_script_entity",
        "request_domain",
        "request_entity",
        "data_leak_type",
    ]
]

df_request_exfiltrators = df_request_exfiltrators.dropna()
df_request_exfiltrators['exfiltrator'] = True

df_event_listeners_wiretappers = df_event_listeners_wiretappers.merge(
    df_request_exfiltrators, how="left", left_on=["site", "script_entity"], right_on=["site", "calling_script_entity"]
) 

df_event_listeners_wiretappers.exfiltrator.fillna(False, inplace=True)

df_event_listeners_wiretappers.rename(columns={"exfiltrator": "leakage"}, inplace=True)

/tmp/ipykernel_628538/1025678490.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_event_listeners_wiretappers.exfiltrator.fillna(False, inplace=True)
/tmp/ipykernel_628538/1025678490.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_event_listeners_wiretappers.exfiltrator.fillna(False, inplace=True

In [17]:
print("Percentage of sites where wiretapping event listeners are installed:", df_event_listeners_wiretappers[['site_domain']].nunique()/n_sites * 100)

Percentage of sites where wiretapping event listeners are installed: site_domain    41.572801
dtype: float64


In [72]:
df_event_listeners_wiretappers.to_parquet("df_event_listeners_wiretappers.parquet", engine="pyarrow", index=False)

In [6]:
df_event_listeners_wiretappers = pd.read_parquet("df_event_listeners_wiretappers.parquet", engine="pyarrow")

In [75]:
df_request_exfils.to_parquet("df_request_exfils.parquet", engine="pyarrow", index=False)

In [10]:
import pyarrow.parquet as pq

table = pq.read_table("df_request_exfils.parquet", use_pandas_metadata=False)
df_request_exfils = table.to_pandas()

TypeError: data type 'dbtime' not understood

In [18]:
df_request_exfils['fl_label'] = df_request_exfils.parallel_apply(label_request_url, args=(filterlists, filterlist_rules), axis=1)

/tmp/ipykernel_628538/3020862868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_request_exfils['fl_label'] = df_request_exfils.parallel_apply(label_request_url, args=(filterlists, filterlist_rules), axis=1)


In [78]:
df_request_exfils.to_parquet("df_request_exfils.parquet", engine="pyarrow", index=False)

In [11]:
import pyarrow as pa
import pyarrow.parquet as pq

# 1) Read the Parquet file _without_ pulling in pandas metadata
table = pq.read_table(
    "df_request_exfils.parquet",
    use_pandas_metadata=False
)

# 2) (Optional) Strip _all_ schema metadata to be 100% sure
table = table.replace_schema_metadata({})

# 3) Drop your unwanted 'time' column (if it still exists)
if "time" in table.schema.names:
    idx = table.schema.get_field_index("time")
    table = table.remove_column(idx)

# 4) Convert to a DataFrame
df_request_exfils = table.to_pandas()

In [19]:
df_script_labels = pd.DataFrame()

# if a script_domain has event 1 fl_label which is equal to True in the df_request_exfils, that script domain label is true

for script_domain in tqdm(df_event_listeners_wiretappers.script_domain.unique()):
    df_script_labels = pd.concat(
        [
            df_script_labels,
            pd.DataFrame(
                {
                    "script_domain": [script_domain],
                    "label": [
                        df_request_exfils[
                            (df_request_exfils.calling_script_domain == script_domain)
                            & (df_request_exfils.fl_label == True)
                        ].shape[0] > 0
                    ],
                }
            ),
        ]
    )

  0%|          | 0/1121 [00:00<?, ?it/s]

In [20]:
print("Average wiretapping event listeners installed per site:", df_event_listeners_wiretappers[['site_domain', 'event_type', 'script_domain']].drop_duplicates().shape[0]/n_sites)

Average wiretapping event listeners installed per site: 0.9514200815423951


In [21]:
print("Percentage of sites where at least one wiretapping event listener is installed:",df_event_listeners_wiretappers.site_domain.nunique() / n_sites * 100)

Percentage of sites where at least one wiretapping event listener is installed: 41.57280077396172


In [22]:
df_wiretappers = df_event_listeners_wiretappers[
    (df_event_listeners_wiretappers.leakage == True)
    & (
        df_event_listeners_wiretappers.script_entity
        == df_event_listeners_wiretappers.request_entity
    )
]

In [23]:
print(
    "Percentage of sites where wiretapping is identified:",
    df_wiretappers[["site_domain"]].nunique() / n_sites * 100,
)

Percentage of sites where wiretapping is identified: site_domain    3.178771
dtype: float64


In [24]:
print("Average number of wiretappers per site:", df_wiretappers[["site_domain", "script_entity"]].drop_duplicates().shape[
    0
] / df_wiretappers[
    ["site_domain"]
].nunique())

Average number of wiretappers per site: site_domain    1.030435
dtype: float64


In [25]:
# add script labels
df_wiretappers = df_wiretappers.merge(
    df_script_labels, how="left", left_on="script_domain", right_on="script_domain"
)

In [26]:
print("Percentage of wiretappers who are also known trackers from filter lists: ",df_wiretappers[(df_wiretappers.label == True)
][["site_domain", "script_entity"]].drop_duplicates().shape[
    0
] / df_wiretappers[
    ["site_domain", "script_entity"]
].drop_duplicates().shape[
    0
] * 100)

Percentage of wiretappers who are also known trackers from filter lists:  84.59915611814345


In [27]:
df_temp = (
    df_wiretappers[["site_domain", "event_type"]]
    .drop_duplicates()
    .event_type.value_counts()
    .reset_index()[:20]
)

df_temp['total_count'] = df_event_listeners_wiretappers[
    ["site_domain", "event_type"]
].drop_duplicates().event_type.value_counts().reset_index()[:20]['count']

df_temp["count_percentage"] = df_temp["count"] / n_sites * 100

df_temp['total_count_percentage'] = df_temp['total_count'] / n_sites * 100

df_temp[["event_type", "total_count_percentage", "count_percentage"]].to_latex(
    "event_types_wiretapping.tex", column_format="lr", float_format="%.2f", index=False
)

In [28]:
import ast


def get_exfiltration_types(script_domain):
    # 1) Filter down to only the rows you care about
    df_filtered = df_requests[
        (df_requests.calling_script_domain.str.contains(script_domain, na=False))
        & (df_requests.data_leak == True)
    ].copy()

    # 2) If data_leak_type is coming in as a string like "['mail','tel']",
    #    turn it into an actual Python list:
    df_filtered["data_leak_type"] = df_filtered["data_leak_type"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # 3) Now explode out the list to one row per single leak-type
    df_exploded = df_filtered.explode("data_leak_type")

    # 4) Drop any null entries (in case some rows had empty lists or None)
    df_exploded = df_exploded[df_exploded.data_leak_type.notnull()]

    replacements = {
        "form_text": "Form Text",
        "form_text2": "Form Text",
        "mail": "Mail",
        "tel": "Phone Number",
        "address": "Address",
        "url_1": "URL",
        "password": "Password",
    }

    df_exploded["data_leak_type"] = df_exploded[
        "data_leak_type"
    ].replace(replacements)

    # 5) If you want the *number of distinct sites* per leak-type:
    result = (
        df_exploded.drop_duplicates(["site_domain", "data_leak_type"])
        .groupby("data_leak_type")["site_domain"]
        .nunique()
        .reset_index(name="site_count")
    )

    # replacements = {
    #     "type_in_body": "Body Text",
    #     "fill_form_url": "Keyboard Interactions",
    #     "fill_form_password": "Keyboard Interactions",
    #     "fill_form_search": "Keyboard Interactions",
    #     "fill_form_email": "Keyboard Interactions",
    #     "fill_form_tel": "Keyboard Interactions",
    #     "fill_form_number": "Keyboard Interactions",
    #     "fill_form_date": "Keyboard Interactions",
    #     "fill_form_text": "Keyboard Interactions",
    #     "fill_form_image": "Keyboard Interactions",
    #     "fill_form_range": "Keyboard Interactions",
    #     "fill_textarea": "Keyboard Interactions",
    #     "pageup_pagedown_tabs": "Keyboard Interactions",
    # }

    # replace

    return ", ".join(result.data_leak_type.unique().tolist())

In [29]:
def get_most_used_events(script_domain):
    return (
        df_event_listeners_wiretappers[
            df_event_listeners_wiretappers.script_domain == script_domain
        ]
        .event_type.value_counts()
        .index.tolist()
    )


def get_most_used_scripts(script_domain):
    return (
        df_event_listeners_wiretappers[
            (df_event_listeners_wiretappers.script_domain == script_domain)
        ]
        .filename.value_counts()
        .head(3)
        .index.tolist()
    )


def get_event_usage(script_domain):
    return (
        df_event_listeners_wiretappers[
            (df_event_listeners_wiretappers.script_domain == script_domain)
        ].site_domain.nunique()
        / n_sites
        * 100
    )

def get_wiretapping_percentage(script_domain):
  return df_wiretappers[(df_wiretappers.script_domain == script_domain)].site_domain.nunique()/n_sites*100


df_event_listeners_stats = df_event_listeners_wiretappers[["site_domain", "script_domain", "event_type"]]

df_script_stats = (
    df_event_listeners_stats[["script_domain", "site_domain"]]
    .drop_duplicates()
    .groupby("script_domain")
    .size()
    .reset_index(name="count")
)

df_top_scripts = (
    df_script_stats.sort_values(by="count", ascending=False)
    .script_domain.head(40)
    .tolist()
)

df_script_info = (
    df_script_stats.sort_values(by="count", ascending=False)
    .head(50)
    .reset_index(drop=True)
    .rename(columns={"script_domain": "Script Domain", "count": "Number of Sites"})
)

df_script_info["event_types"] = df_script_info["Script Domain"].progress_apply(
    lambda x: ", ".join(get_most_used_events(x))
)

df_script_info["scripts_used"] = df_script_info["Script Domain"].progress_apply(
    lambda x: ", ".join(get_most_used_scripts(x))
)

df_script_info["wiretapping_percentage"] = df_script_info[
    "Script Domain"
].progress_apply(get_wiretapping_percentage)

df_script_info["Prevelance on Sites"] = df_script_info["Script Domain"].progress_apply(
    get_event_usage
)

df_script_info["Percentage of Sites"] = df_script_info["Number of Sites"] / 14471 * 100

# round to 2 decimal places
df_script_info["Percentage of Sites"] = df_script_info["Percentage of Sites"].round(3)

# round all the percentages to 2 decimal places
df_script_info["wiretapping_percentage"] = df_script_info[
    "wiretapping_percentage"
].round(3)

# add script labels
df_script_info = df_script_info.merge(
    df_script_labels, how="left", left_on="Script Domain", right_on="script_domain"
)

df_script_info.rename({'label': 'known_tracker'}, axis=1, inplace=True)

df_script_info["is_wiretapping"] = df_script_info["wiretapping_percentage"] > 0

df_script_info["Data Shared"] = df_script_info["Script Domain"].progress_apply(
    get_exfiltration_types
)

df_script_info[
    [
        "Script Domain",
        "Prevelance on Sites",
        "wiretapping_percentage",
        "known_tracker",
        "is_wiretapping",
        "event_types",
        "Data Shared",
        "scripts_used",
    ]
].sort_values('Prevelance on Sites', ascending=False).to_latex(
    "wiretappers_complete.tex",
    index=False,
    column_format="lrrlll",
    escape=True,
    float_format="%.2f",
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [31]:
import ast


def get_exfiltration_types_site_domain(script_domain, site_domain):
    # 1) Filter down to only the rows you care about
    df_filtered = df_requests[
        (df_requests.calling_script_domain.str.contains(script_domain, na=False))
        & (df_requests.data_leak == True)
        & (df_requests.site_domain == site_domain)
    ].copy()

    # 2) If data_leak_type is coming in as a string like "['mail','tel']",
    #    turn it into an actual Python list:
    df_filtered["data_leak_type"] = df_filtered["data_leak_type"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    # 3) Now explode out the list to one row per single leak-type
    df_exploded = df_filtered.explode("data_leak_type")

    # 4) Drop any null entries (in case some rows had empty lists or None)
    df_exploded = df_exploded[df_exploded.data_leak_type.notnull()]

    replacements = {
        "form_text": "Form Text",
        "form_text2": "Form Text",
        "mail": "Mail",
        "tel": "Phone Number",
        "address": "Address",
        "url_1": "URL",
        "password": "Password",
    }

    df_exploded["data_leak_type"] = df_exploded[
        "data_leak_type"
    ].replace(replacements)

    # 5) If you want the *number of distinct sites* per leak-type:
    result = (
        df_exploded.drop_duplicates(["site_domain", "data_leak_type"])
        .groupby("data_leak_type")["site_domain"]
        .nunique()
        .reset_index(name="site_count")
    )

    # replacements = {
    #     "type_in_body": "Body Text",
    #     "fill_form_url": "Keyboard Interactions",
    #     "fill_form_password": "Keyboard Interactions",
    #     "fill_form_search": "Keyboard Interactions",
    #     "fill_form_email": "Keyboard Interactions",
    #     "fill_form_tel": "Keyboard Interactions",
    #     "fill_form_number": "Keyboard Interactions",
    #     "fill_form_date": "Keyboard Interactions",
    #     "fill_form_text": "Keyboard Interactions",
    #     "fill_form_image": "Keyboard Interactions",
    #     "fill_form_range": "Keyboard Interactions",
    #     "fill_textarea": "Keyboard Interactions",
    #     "pageup_pagedown_tabs": "Keyboard Interactions",
    # }

    # replace

    return result.data_leak_type.unique().tolist()

In [33]:
import ast
import re
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# --- 1) Precompute a tidy, exploded table once ---
req = df_requests.loc[df_requests["data_leak"] == True, [
    "site_domain", "calling_script_domain", "data_leak_type"
]].copy()

# parse list-like strings once
mask_str = req["data_leak_type"].apply(lambda x: isinstance(x, str))
req.loc[mask_str, "data_leak_type"] = req.loc[mask_str, "data_leak_type"].progress_apply(ast.literal_eval)

# explode once
req = req.explode("data_leak_type")
req = req[req["data_leak_type"].notnull()]

# normalize labels once
replacements = {
    "form_text": "Form Text",
    "form_text2": "Form Text",
    "mail": "Mail",
    "tel": "Phone Number",
    "address": "Address",
    "url_1": "URL",
    "password": "Password",
}
req["data_leak_type"] = req["data_leak_type"].replace(replacements)

# one row per (script, site, type)
agg = (
    req.drop_duplicates(["calling_script_domain", "site_domain", "data_leak_type"])
      .groupby(["calling_script_domain", "site_domain"])["data_leak_type"]
      .apply(lambda s: sorted(s.unique()))
      .reset_index(name="data_shared")
)

# --- 2) Merge onto your target rows (exact match) ---
df_out = (
    df_event_listeners_wiretappers[["site_domain", "script_domain", "event_type"]]
      .drop_duplicates()
      .merge(agg, left_on=["script_domain", "site_domain"],
                  right_on=["calling_script_domain", "site_domain"],
             how="left")
      .drop(columns=["calling_script_domain"])
)

# If you want an empty list instead of NaN:
df_out["data_shared"] = df_out["data_shared"].apply(lambda x: x if isinstance(x, list) else [])

100%|██████████| 131343/131343 [00:00<00:00, 174076.68it/s]


In [38]:
# explode the data_shared column
df_out = df_out.explode("data_shared")

In [48]:
#drop rows where data_shared is empty
df_out = df_out[df_out.data_shared.notna()]

# calculate statistic that which script_domain is sharing what type of data on percentage of sites, here we should group by script_domain and data_shared
df_stats = df_out[['site_domain', 'data_shared']].drop_duplicates().groupby(["data_shared"]).size().reset_index(name="count")
df_stats["percentage"] = df_stats["count"] / n_sites * 100

In [53]:
df_out.site_domain.nunique()/n_sites * 100

4.961647432796628

In [49]:
df_stats.sort_values(by=["percentage"], inplace=True, ascending=False)

In [52]:
df_stats[['data_shared', 'percentage']].to_latex("data_shared_stats.tex", index=False, column_format="llr", float_format="%.2f")

In [31]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import ast


def create_gannt_chart(script_domain, site, event_name, max_seconds=90):
    df_keydown = df_event_listeners[
        df_event_listeners.script_domain.str.contains(script_domain, na=False)
        & (df_event_listeners.event_type.str.contains(event_name, na=False))
    ]

    df_events = df_keydown[df_keydown.site.str.contains(site, na=False)]

    df_script_exfils = df_request_exfils[df_request_exfils.site.str.contains(site)]

    df_script_exfils = df_script_exfils[
        (
            (df_script_exfils.request_domain.str.contains(script_domain))
            | (df_script_exfils.calling_script_domain.str.contains(script_domain))
        )
    ][["request_time", "data_leak_type"]]

    df_script_exfils['data_leak_type'] = df_script_exfils['data_leak_type'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    df_script_exfils = df_script_exfils.explode('data_leak_type')

    df_script_exfils = df_script_exfils[df_script_exfils.data_leak_type.notnull()]

    # df_script_exfils['data_leak_type'] = df_script_exfils['data_leak_type'].apply(lambda x: x['Type'] if x is not None else pd.NA)
    # df_script_exfils['matched_sensitive_category'] = df_script_exfils['data_leak_type'].apply(lambda x: x['Category'] if x is not None else pd.NA)

    df_script_exfils = df_script_exfils[['request_time', 'data_leak_type']].drop_duplicates()

    # print(df_events.measurement.value_counts())

    # df_events = df_events[df_events.measurement.str.contains('new')]
    df_events = df_events[["event_time", "init_invoke"]]

    df_site_interactions = df_interactions[df_interactions.site_url.str.contains(site)]

    # print(df_site_interactions.measurement.value_counts())

    # df_site_interactions = df_site_interactions[df_site_interactions.measurement.str.contains('new')]

    df_site_interactions = df_site_interactions[
        ["start_time", "end_time", "interaction_type"]
    ]

    # df_events["event_time"] = pd.to_datetime(df_events["event_time"])
    df_events["event_time"] = (
        pd.to_datetime(df_events["event_time"], utc=True).dt.tz_localize(None)
    )

    df_script_exfils = df_script_exfils.rename(columns={'request_time': 'event_time'})

    # df_script_exfils["event_time"] = pd.to_datetime(df_script_exfils["event_time"])
    df_script_exfils["event_time"] = (
        pd.to_datetime(df_script_exfils["event_time"], utc=True)
        .dt.tz_localize(None)
    )

    # df_site_interactions["start_time"] = pd.to_datetime(
    #     df_site_interactions["start_time"]
    # )
    # df_site_interactions["end_time"] = pd.to_datetime(df_site_interactions["end_time"])

    df_site_interactions["start_time"] = (
        pd.to_datetime(df_site_interactions["start_time"], utc=True)
        .dt.tz_localize(None)
    )
    df_site_interactions["end_time"] = (
        pd.to_datetime(df_site_interactions["end_time"], utc=True)
        .dt.tz_localize(None)
    )

    # smallest event time in df_ekon
    min_event = df_events.event_time.min()

    # smallest event time in df_ekon_interactions
    min_interactions = df_site_interactions.start_time.min()

    min_event_naive = min_event.tz_localize(None)
    min_interactions_naive = min_interactions.tz_localize(None)
    print(min_event_naive, min_interactions_naive)
    min_timestamp = min(min_event_naive, min_interactions_naive)

    df_events["event_time"] = df_events["event_time"] - min_timestamp

    df_script_exfils["event_time"] = df_script_exfils["event_time"] - min_timestamp

    df_site_interactions["start_time"] = (
        df_site_interactions["start_time"].dt.tz_localize(None) - min_timestamp
    )
    df_site_interactions["end_time"] = (
        df_site_interactions["end_time"].dt.tz_localize(None) - min_timestamp
    )

    # # convert to seconds start_time and end_time
    df_site_interactions["start_time"] = df_site_interactions[
        "start_time"
    ].dt.total_seconds()
    df_site_interactions["end_time"] = df_site_interactions[
        "end_time"
    ].dt.total_seconds()
    df_events["event_time"] = df_events["event_time"].dt.total_seconds()

    df_script_exfils["event_time"] = df_script_exfils["event_time"].dt.total_seconds()

    # remove the negative values

    print(df_script_exfils.head())

    df_script_exfils = df_script_exfils[df_script_exfils.event_time > 0]

    # min_exfil_seconds = df_script_exfils.event_time.min()

    # print(f"Min exfil seconds: {min_exfil_seconds}")

    max_exfil_seconds = df_script_exfils.event_time.max()
    max_interaction_seconds = df_site_interactions.end_time.max()
    max_event_seconds = df_events.event_time.max()

    # if min_exfil_seconds > max_seconds:
    #     max_seconds = min_exfil_seconds * 2
    #     print(f"Max seconds: {max_seconds}")

    max_seconds = min(max_exfil_seconds, max_interaction_seconds, max_event_seconds, max_seconds) + 2

    interactions = df_site_interactions.interaction_type.unique().tolist()

    # df_script_exfils = df_script_exfils[df_script_exfils.event_time < max_seconds]

    print(df_script_exfils.sort_values(by='event_time'))

    replacements = {
        "type_in_body": "Keyboard Interactions",
        "fill_form_url": "Keyboard Interactions",
        "fill_form_password": "Keyboard Interactions",
        "fill_form_search": "Keyboard Interactions",
        "fill_form_email": "Keyboard Interactions",
        "fill_form_tel": "Keyboard Interactions",
        "fill_form_number": "Keyboard Interactions",
        "fill_form_date": "Keyboard Interactions",
        "fill_form_text": "Keyboard Interactions",
        "fill_form_image": "Keyboard Interactions",
        "fill_form_range": "Keyboard Interactions",
        "fill_textarea": "Keyboard Interactions",
        "pageup_pagedown_tabs": "Keyboard Interactions",
    }

    removals = ["mouse_movement"]

    df_site_interactions = df_site_interactions[
        ~df_site_interactions.interaction_type.isin(removals)
    ]

    df_site_interactions["interaction_type"] = df_site_interactions[
        "interaction_type"
    ].replace(replacements)

    df_site_interactions["duration"] = (
        df_site_interactions.end_time - df_site_interactions.start_time
    )

    fig, ax = plt.subplots(figsize=(10, 4))

    interaction_colors = {
        "Keyboard Interactions": "green",
        # "Type in Field": "blue",
        # "Navigate using Keys": "purple",
    }

    # Define rows with y-positions, adding a dedicated row for invocations
    interaction_labels = [
        "Keyboard Interactions",
        # "Type in Field",
        # "Navigate using Keys",
        "Invocations",
    ]
    y_positions = {label: i for i, label in enumerate(interaction_labels)}

    # Plot the timeline bars for "Type on Webpage" and "Type in Field" rows only
    for index, row in df_site_interactions.iterrows():
        y_pos = y_positions[row["interaction_type"]]
        ax.barh(
            y=y_pos,
            width=row["duration"],
            left=row["start_time"],
            color=interaction_colors[row["interaction_type"]],
        )

    # Set the title and invert the y-axis
    # ax.set_title(
    #     f"Timeline of keydown events on {get_domain(site)} for {script_domain} scripts",
    #     fontsize=12,
    # )
    plt.gca().invert_yaxis()

    # Set custom y-ticks for the rows
    ax.set_yticks(list(y_positions.values()))
    ax.set_yticklabels(interaction_labels, fontsize=12)

    # Place invocation lines (red and orange) in the "Invocations" row only
    for index, row in df_events.iterrows():
        color = "blue" if row["init_invoke"] == "init" else "orange"
        # Use the y-coordinate of the "Invocations" row only
        if row["init_invoke"] == "invoke":
            ax.axvline(
                x=row["event_time"], color=color, linestyle="solid", ymin=0, ymax=0.3
            )
        else:
            ax.axvline(
                x=row["event_time"], color=color, linestyle="dotted", linewidth=1.5
            )

    for index, row in df_script_exfils.iterrows():
        ax.axvline(x=row["event_time"], color="red", linestyle="solid", linewidth=1.5)

    # Limit the x-axis to the first 100 seconds
    ax.set_xlim(-0.5, max_seconds)

    # Custom legend for timeline bars and invocation lines
    interaction_patches = [
        mpatches.Patch(color=interaction_colors[k], label=k) for k in interaction_colors
    ]
    line_patches = [
        mpatches.Patch(color="blue", label="Listener Initialized"),
        mpatches.Patch(color="orange", label="Listener Invoked"),
        mpatches.Patch(color="red", label="Shared to Remote Server"),
    ]

    # Add the legend and adjust layout
    ax.legend(
        handles=interaction_patches + line_patches,
        fontsize=12,
        loc="upper left",
        bbox_to_anchor=(1, 1),
    )
    # Add x-axis label
    ax.set_xlabel("Time (seconds)", fontsize=12)

    ax.xaxis.grid(True)

    plt.tight_layout(rect=[0, 0, 0.85, 1])

    # Save the figure as a PDF
    plt.savefig(f"{script_domain}_{get_domain(site)}_{event_name}.pdf", dpi=300)
    # plt.show()

In [32]:
df_wiretappers[
    df_wiretappers.script_domain.str.contains("google.com", na=False)
].site.value_counts()[
    :40
]

site
https://docs.samsungknox.com/admin                                                         224
https://docs.samsungknox.com/admin/index.html                                              176
https://phileweb.com                                                                        91
https://aerospaceweb.org/aircraft/                                                          80
https://docs.samsungknox.com/admin/knox-authentication-manager/welcome.htm                  72
https://aerospaceweb.org/about/                                                             60
https://aerospaceweb.org/about/copyright.shtml                                              60
https://www.openculture.com/2010/11/donald_duck_wants_you_to_pay_your_taxes_1943.html       56
https://www.openculture.com/2007/08/life-changing_books_your_picks.html                     56
https://www.vbforums.com/announcement.php?f=7&a=107/                                        56
https://www.openculture.com/2006/10/foreign_l

In [34]:
create_gannt_chart(
    "google.com",
    "https://phileweb.com",
    "keydown",
)

2025-05-01 18:04:03.367000 2025-05-01 18:04:16.560000
          event_time data_leak_type
344282        18.383      form_text
5971533       18.067      form_text
6545223       18.471      form_text
6791340       18.667      form_text
16020581      18.559      form_text
          event_time data_leak_type
5971533       18.067      form_text
21317222      18.176      form_text
18122000      18.282      form_text
344282        18.383      form_text
6545223       18.471      form_text
16020581      18.559      form_text
6791340       18.667      form_text


In [ ]:
df_event_listeners_wiretappers.head()

In [151]:
df_category = pd.read_csv('category_data.csv')

In [152]:
df_category

,site_id,site,url,current_etld,target_etld,Type,Category
0,127613_4,https://dailyinfographic.com/15-coffee-facts,https://api.rlcdn.com/api/identity/v2/envelope...,dailyinfographic.com,rlcdn.com,SHA1,mail
1,127613_4,https://dailyinfographic.com/15-coffee-facts,https://api.rlcdn.com/api/identity/v2/envelope...,dailyinfographic.com,rlcdn.com,SHA256,mail
2,162512_4,https://datum-group.ru/contacts/,https://mc.yandex.com/webvisor/23353483?wv-par...,datum-group.ru,yandex.com,Plain,form_text
3,162512_4,https://datum-group.ru/contacts/,https://mc.yandex.com/webvisor/23353483?wv-par...,datum-group.ru,yandex.com,URLEncode,form_text
4,298026_6,https://www.loccitane.ru/best-travel-companion/,https://mc.yandex.com/webvisor/10543693?wv-par...,loccitane.ru,yandex.com,Plain,form_text
...,...,...,...,...,...,...,...
10161,417594_9,https://ohiochannel.org/collections/conference...,https://www.google-analytics.com/g/collect?v=2...,ohiochannel.org,google-analytics.com,URLEncode,form_text
10162,801348_4,https://myfittingroom.dk/shop/gavekort-54c1.html,https://api.clerk.io/v3/search/omni?payload=%7...,myfittingroom.dk,clerk.io,Plain,form_text
10163,801348_4,https://myfittingroom.dk/shop/gavekort-54c1.html,https://api.clerk.io/v3/search/omni?payload=%7...,myfittingroom.dk,clerk.io,URLEncode,form_text
10164,838563_10,https://www.explorescientific.com/collections/...,https://www.facebook.com/tr/?id=19708551834599...,explorescientific.com,facebook.com,Plain,phone


In [153]:
df_category.shape

(10166, 7)

In [154]:
df_category = df_category[["current_etld", "target_etld", "Category"]].drop_duplicates()

In [155]:
df_category.Category.value_counts()

Category
form_text    722
mail         291
phone         24
Name: count, dtype: int64

In [150]:
df_event_listeners_wiretappers[df_event_listeners_wiretappers.leakage == True].head()

,site,event_type,event_time,init_invoke,stack,measurement,url,filename,script_domain,site_domain,request_url,stack_json,script_entity,site_entity,calling_script_filename,calling_script_domain,exfiltrator,leakage
313,https://kinotv.ru/movies-collections/apple-tv/,keyup,2024-10-14 05:46:31.487,invoke,Error\n at window.addEventListener (<anonym...,old,https://mc.yandex.ru/metrika/tag.js:140:396),tag.js,yandex.ru,kinotv.ru,NaN,NaN,Yandex LLC,kinotv.ru,tag.js,yandex.ru,True,True
314,https://peleng.by/service,keyup,2024-10-16 00:40:07.347,invoke,Error\n at window.addEventListener (<anonym...,old,https://mc.yandex.ru/metrika/tag.js:140:396),tag.js,yandex.ru,peleng.by,NaN,NaN,Yandex LLC,peleng.by,tag.js,yandex.ru,True,True
321,https://ikp-rao.ru/intervju-ikp-live/,keyup,2024-10-15 10:07:19.322,invoke,Error\n at window.addEventListener (<anonym...,old,https://mc.yandex.ru/metrika/tag.js:140:396),tag.js,yandex.ru,ikp-rao.ru,NaN,NaN,Yandex LLC,ikp-rao.ru,tag.js,yandex.ru,True,True
326,https://omsk.sskindustry.ru/,keyup,2024-10-14 04:53:44.814,invoke,Error\n at window.addEventListener (<anonym...,old,https://mc.yandex.ru/metrika/tag.js:140:396),tag.js,yandex.ru,sskindustry.ru,NaN,NaN,Yandex LLC,sskindustry.ru,tag.js,yandex.ru,True,True
329,https://nelvamoda.by/collection/zhilety-i-nakidki,keyup,2024-10-15 16:46:51.890,invoke,Error\n at window.addEventListener (<anonym...,old,https://mc.yandex.ru/metrika/tag.js:140:396),tag.js,yandex.ru,nelvamoda.by,NaN,NaN,Yandex LLC,nelvamoda.by,tag.js,yandex.ru,True,True


In [156]:
df_requests_exfils = df_requests

In [157]:
df_requests_exfils.shape

(32544214, 17)

In [158]:
df_requests_exfils

,site,url,method,request_time,after_interaction,data_leak,time,matched_sensitive_patterns,measurement,request_call_stack,calling_script,calling_script_filename,site_domain,request_domain,calling_script_domain,request_entity,calling_script_entity
0,https://getresq.com,https://gs-cdn.optimonk.com/jfclientsdk/latest...,GET,2024-10-15 15:41:01.320,<NA>,False,NaT,[],old,NaN,<NA>,<NA>,getresq.com,optimonk.com,None,WebShop Marketing Kft.,None
1,https://npsgk.ru/division-railways/,https://npsgk.ru/local/templates/nps/assets/32...,GET,2024-10-16 07:12:11.959,<NA>,False,NaT,[],old,NaN,<NA>,<NA>,npsgk.ru,npsgk.ru,None,npsgk.ru,None
2,https://firstdue.com,https://cdn.prod.website-files.com/5e4f0520105...,GET,2024-10-15 18:12:10.036,<NA>,False,NaT,[],old,NaN,<NA>,<NA>,firstdue.com,website-files.com,None,"Webflow, Inc.",None
3,https://highlandscurrent.org/2024/08/09/mudlar...,https://s.clarity.ms/collect,POST,2024-10-14 09:34:58.260,<NA>,False,NaT,[],old,NaN,<NA>,<NA>,highlandscurrent.org,clarity.ms,None,Microsoft Corporation,None
4,https://community-slack.chef.io/,https://www.facebook.com/privacy_sandbox/pixel...,GET,2024-10-16 01:17:27.630,<NA>,False,NaT,[],old,NaN,<NA>,<NA>,chef.io,facebook.com,None,"Facebook, Inc.",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927200,https://ionwave.net/bid-tabulation-excel-examp...,https://b.6sc.co/v1/beacon/img.gif?token=db473...,GET,2024-11-08 21:09:45.596,<NA>,False,21:09:45.596000,[],new,"{""type"":""other""}",<NA>,<NA>,ionwave.net,6sc.co,None,6 Sense Insights Inc.,None
1927201,https://blog.enscape3d.com/content-hub/stories...,https://track.hubspot.com/__ptq.gif?k=18&fi=13...,GET,2024-11-08 22:06:58.995,<NA>,False,22:06:58.995000,[],new,"{""type"":""other""}",<NA>,<NA>,enscape3d.com,hubspot.com,None,"HubSpot, Inc.",None
1927202,https://myspace.com/article/2022/2/22/euphoria...,https://a4-videos.myspacecdn.com/videos02/173/...,GET,2024-11-08 21:20:45.888,<NA>,False,21:20:45.888000,[],new,"{""type"":""other""}",<NA>,<NA>,myspace.com,myspacecdn.com,None,MySpace LLC,None
1927203,https://www.smashingmagazine.com/2020/03/smart...,https://www.smashingmagazine.com/js/AddToCart....,GET,2024-11-08 22:31:00.259,<NA>,False,22:31:00.259000,[],new,"{""type"":""script"",""stack"":{""callFrames"":[{""func...",https://www.smashingmagazine.com/js/app.js,app.js,smashingmagazine.com,smashingmagazine.com,smashingmagazine.com,smashingmagazine.com,smashingmagazine.com


In [159]:
df_requests_exfils = df_requests_exfils[['site_domain', 'request_domain', 'calling_script_domain', 'calling_script_filename']].drop_duplicates()

In [160]:
df_requests_exfils = pd.merge(
    df_requests_exfils,
    df_category,
    how="left",
    left_on=["site_domain", "request_domain"],
    right_on=["current_etld", "target_etld"],
)

In [162]:
df_requests_exfils.Category.value_counts()

Category
form_text    2299
mail          954
phone          78
Name: count, dtype: int64

In [163]:
df_events_exfils = df_event_listeners_wiretappers[df_event_listeners_wiretappers.leakage == True][['site_domain', 'script_domain', 'filename']]

In [164]:
df_events_exfils.drop_duplicates(inplace=True)

In [166]:
df_requests_exfils.head()

,site_domain,request_domain,calling_script_domain,calling_script_filename,current_etld,target_etld,Category
0,getresq.com,optimonk.com,None,<NA>,NaN,NaN,NaN
1,npsgk.ru,npsgk.ru,None,<NA>,NaN,NaN,NaN
2,firstdue.com,website-files.com,None,<NA>,NaN,NaN,NaN
3,highlandscurrent.org,clarity.ms,None,<NA>,NaN,NaN,NaN
4,chef.io,facebook.com,None,<NA>,NaN,NaN,NaN


In [168]:
df_events_exfils = pd.merge(
    df_events_exfils,
    df_requests_exfils[['site_domain', 'calling_script_domain', 'Category']].drop_duplicates(),
    how="left",
    left_on=["site_domain", "script_domain"],
    right_on=["site_domain", "calling_script_domain"],
)

In [136]:
df_events_exfils['site_domain'] = df_events_exfils.site.apply(get_domain)

In [169]:
df_events_exfils = df_events_exfils[['site_domain', 'script_domain', 'Category']].drop_duplicates()

In [176]:
df_counts = df_events_exfils.groupby(["script_domain", "Category"]).size().reset_index(
    name="counts"
).sort_values(by="counts", ascending=False)

In [178]:
df_counts['percentage'] = df_counts['counts'] / 25000 * 100

In [179]:
df_counts.replace("form_text", "Form Text", inplace=True)
df_counts.replace("mail", "Email", inplace=True)
df_counts.replace("phone", "Phone Number", inplace=True)

In [183]:
df_counts[:20][['script_domain', 'Category', 'percentage']].to_latex('top_scripts_categories.tex', index=False, column_format='lcr', float_format="%.2f")